In [9]:
# 区域卷积神经⽹络（R-CNN）系列
''' 
除了 13.7节中描述的单发多框检测之外，区域卷积神经⽹络（region-based CNN或regions with CNN features，
R-CNN）[Girshick et al., 2014]也是将深度模型应⽤于⽬标检测的开创性⼯作之⼀。在本节中，我们将介绍R-
CNN及其⼀系列改进⽅法：快速的R-CNN（Fast R-CNN）[Girshick, 2015]、更快的R-CNN（Faster R-CNN）
[Ren et al., 2015]和掩码R-CNN（Mask R-CNN）[He et al., 2017]。限于篇幅，我们只着重介绍这些模型的设
计思路
----------------------------------------------------------------------------------------------
R-CNN

R-CNN⾸先从输⼊图像中选取若⼲（例如2000个）提议区域（如锚框也是⼀种选取⽅法），并标注它们的类别
和边界框（如偏移量）。[Girshick et al., 2014]然后，⽤卷积神经⽹络对每个提议区域进⾏前向传播以抽取其
特征。接下来，我们⽤每个提议区域的特征来预测类别和边界框

图13.8.1展⽰了R-CNN模型。具体来说，R-CNN包括以下四个步骤：
1. 对输⼊图像使⽤选择性搜索来选取多个⾼质量的提议区域 [Uijlings et al., 2013]。这些提议区域通常是
在多个尺度下选取的，并具有不同的形状和⼤⼩。每个提议区域都将被标注类别和真实边界框。
2. 选择⼀个预训练的卷积神经⽹络，并将其在输出层之前截断。将每个提议区域变形为⽹络需要的输⼊
尺⼨，并通过前向传播输出抽取的提议区域特征。
3. 将每个提议区域的特征连同其标注的类别作为⼀个样本。训练多个⽀持向量机对⽬标分类，其中每个
⽀持向量机⽤来判断样本是否属于某⼀个类别。
4. 将每个提议区域的特征连同其标注的边界框作为⼀个样本，训练线性回归模型来预测真实边界框。
尽管R-CNN模型通过预训练的卷积神经⽹络有效地抽取了图像特征，但它的速度很慢。想象⼀下，我们可能
从⼀张图像中选出上千个提议区域，这需要上千次的卷积神经⽹络的前向传播来执⾏⽬标检测。这种庞⼤的
计算量使得R-CNN在现实世界中难以被⼴泛应⽤
----------------------------------------------------------------------------------------------
Fast R-CNN

R-CNN的主要性能瓶颈在于，对每个提议区域，卷积神经⽹络的前向传播是独⽴的，⽽没有共享计算。由于
这些区域通常有重叠，独⽴的特征抽取会导致重复的计算。Fast R-CNN [Girshick, 2015]对R-CNN的主要改进
之⼀，是仅在整张图象上执⾏卷积神经⽹络的前向传播

图13.8.2中描述了Fast R-CNN模型。它的主要计算如下：
1. 与R-CNN相⽐，Fast R-CNN⽤来提取特征的卷积神经⽹络的输⼊是整个图像，⽽不是各个提议区域。此
外，这个⽹络通常会参与训练。设输⼊为⼀张图像，将卷积神经⽹络的输出的形状记为1 × c × h1 × w1。
2. 假设选择性搜索⽣成了n个提议区域。这些形状各异的提议区域在卷积神经⽹络的输出上分别标出了形
状各异的兴趣区域。然后，这些感兴趣的区域需要进⼀步抽取出形状相同的特征（⽐如指定⾼度h2和宽
度w2），以便于连结后输出。为了实现这⼀⽬标，Fast R-CNN引⼊了兴趣区域汇聚层（RoI pooling）：将卷
积神经⽹络的输出和提议区域作为输⼊，输出连结后的各个提议区域抽取的特征，形状为n×c×h2×w2。
3. 通过全连接层将输出形状变换为n × d，其中超参数d取决于模型设计。
4. 预测n个提议区域中每个区域的类别和边界框。更具体地说，在预测类别和边界框时，将全连接层的输出
分别转换为形状为n×q（q是类别的数量）的输出和形状为n×4的输出。其中预测类别时使⽤softmax回
归。

在Fast R-CNN中提出的兴趣区域汇聚层与 6.5节中介绍的汇聚层有所不同。在汇聚层中，我们通过设置汇聚
窗⼝、填充和步幅的⼤⼩来间接控制输出形状。⽽兴趣区域汇聚层对每个区域的输出形状是可以直接指定的。
例如，指定每个区域输出的⾼和宽分别为h2和w2。对于任何形状为h × w的兴趣区域窗⼝，该窗⼝将被划分
为h2 × w2⼦窗⼝⽹格，其中每个⼦窗⼝的⼤⼩约为(h/h2) × (w/w2)。在实践中，任何⼦窗⼝的⾼度和宽度
都应向上取整，其中的最⼤元素作为该⼦窗⼝的输出。因此，兴趣区域汇聚层可从形状各异的兴趣区域中均
抽取出形状相同的特征。

作为说明性⽰例，图13.8.3中提到，在4 × 4的输⼊中，我们选取了左上⻆3 × 3的兴趣区域。对于该兴趣区域，
我们通过2 × 2的兴趣区域汇聚层得到⼀个2 × 2的输出。请注意，四个划分后的⼦窗⼝中分别含有元素0、1、
4、5（5最⼤）；2、6（6最⼤）；8、9（9最⼤）；以及10

下⾯，我们演⽰了兴趣区域汇聚层的计算⽅法。假设卷积神经⽹络抽取的特征X的⾼度和宽度都是4，且只有
单通道
'''
import torch
import torchvision

X = torch.arange(16.).reshape(1, 1, 4, 4)
X
X[:, :, 1:4, 0:4]

tensor([[[[ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [7]:
''' 
让我们进⼀步假设输⼊图像的⾼度和宽度都是40像素，且选择性搜索在此图像上⽣成了两个提议区域。每个
区域由5个元素表⽰：区域⽬标类别、左上⻆和右下⻆的(x, y)坐标
'''
# boxes (Tensor[K, 5] or List[Tensor[L, 4]]): the box coordinates in (x1, y1, x2, y2)
# 对应 X[:, :, 0:3, 0:3]和X[:, :, 1:4, 0:4]
rois = torch.Tensor([[0, 0, 0, 20, 20], [0, 0, 10, 30, 30]]) # 两个提议区域

In [8]:
''' 
由于X的⾼和宽是输⼊图像⾼和宽的1/10，因此，两个提议区域的坐标先按spatial_scale乘以0.1。然后，
在X上分别标出这两个兴趣区域X[:, :, 0:3, 0:3]和X[:, :, 1:4, 0:4]。最后，在2 × 2的兴趣区
域汇聚层中，每个兴趣区域被划分为⼦窗⼝⽹格，并进⼀步抽取相同形状2 × 2的特征

spatial  空间
RoI  Region of interest pooling  兴趣区域池化 向上取整

roi_pool(
    input: Tensor,
    boxes: Union[Tensor, List[Tensor]],
    output_size: BroadcastingList2[int],
    spatial_scale: float = 1.0,
)
'''
torchvision.ops.roi_pool(X, rois, output_size=(2, 2), spatial_scale=0.1) # RoI pooling

tensor([[[[ 5.,  6.],
          [ 9., 10.]]],


        [[[ 9., 11.],
          [13., 15.]]]])

In [ ]:
'''
Faster R-CNN

为了较精确地检测⽬标结果，Fast R-CNN模型通常需要在选择性搜索中⽣成⼤量的提议区域。Faster R-CNN
[Ren et al., 2015]提出将选择性搜索替换为区域提议⽹络（region proposal network），从⽽减少提议区域的
⽣成数量，并保证⽬标检测的精度  proposal 提议

图13.8.4描述了Faster R-CNN模型。与Fast R-CNN相⽐，Faster R-CNN只将⽣成提议区域的⽅法从选择性搜
索改为了区域提议⽹络，模型的其余部分保持不变。具体来说，区域提议⽹络的计算步骤如下：
1. 使⽤填充为1的3 × 3的卷积层变换卷积神经⽹络的输出，并将输出通道数记为c。这样，卷积神经⽹络
为图像抽取的特征图中的每个单元均得到⼀个⻓度为c的新特征。
2. 以特征图的每个像素为中⼼，⽣成多个不同⼤⼩和宽⾼⽐的锚框并标注它们。
3. 使⽤锚框中⼼单元⻓度为c的特征，分别预测该锚框的⼆元类别（含⽬标还是背景）和边界框。
4. 使⽤⾮极⼤值抑制，从预测类别为⽬标的预测边界框中移除相似的结果。最终输出的预测边界框即是
兴趣区域汇聚层所需的提议区域

值得⼀提的是，区域提议⽹络作为Faster R-CNN模型的⼀部分，是和整个模型⼀起训练得到的。换句话说，
Faster R-CNN的⽬标函数不仅包括⽬标检测中的类别和边界框预测，还包括区域提议⽹络中锚框的⼆元类别
和边界框预测。作为端到端训练的结果，区域提议⽹络能够学习到如何⽣成⾼质量的提议区域，从⽽在减少
了从数据中学习的提议区域的数量的情况下，仍保持⽬标检测的精度
----------------------------------------------------------------------------------------------
Mask R-CNN

如果在训练集中还标注了每个⽬标在图像上的像素级位置，那么Mask R-CNN [He et al., 2017]能够有效地利
⽤这些详尽的标注信息进⼀步提升⽬标检测的精度。

如 图13.8.5所⽰，Mask R-CNN是基于Faster R-CNN修改⽽来的。具体来说，Mask R-CNN将兴趣区域汇聚层
替换为了 兴趣区域对⻬层，使⽤双线性插值（bilinear interpolation）来保留特征图上的空间信息，从⽽更
适于像素级预测。兴趣区域对⻬层的输出包含了所有与兴趣区域的形状相同的特征图。它们不仅被⽤于预测
每个兴趣区域的类别和边界框，还通过额外的全卷积⽹络预测⽬标的像素级位置。本章的后续章节将更详细
地介绍如何使⽤全卷积⽹络预测图像中像素级的语义
----------------------------------------------------------------------------------------------
⼩结
• R-CNN对图像选取若⼲提议区域，使⽤卷积神经⽹络对每个提议区域执⾏前向传播以抽取其特征，然
后再⽤这些特征来预测提议区域的类别和边界框。
• Fast R-CNN对R-CNN的⼀个主要改进：只对整个图像做卷积神经⽹络的前向传播。它还引⼊了兴趣区
域汇聚层(RoI)，从⽽为具有不同形状的兴趣区域抽取相同形状的特征。
• Faster R-CNN将Fast R-CNN中使⽤的选择性搜索替换为参与训练的区域提议⽹络，这样后者可以在减
少提议区域数量的情况下仍保证⽬标检测的精度。
• Mask R-CNN在Faster R-CNN的基础上引⼊了⼀个全卷积⽹络，从⽽借助⽬标的像素级位置进⼀步提升
⽬标检测的精度
'''